# MVVM + Hexagonal Architecture in SEVA — Part 2 (Views & ViewModels)

This notebook continues **Part 1** and focuses on **Views** and **ViewModels**:
- What each layer *is allowed to do*.
- How they are **initialized** in a clean, testable way.
- How **modularity** is achieved (swapping adapters, mock use cases, and dependency injection).

> If you haven’t seen Part 1, start there: it introduces the Ports/Adapters workflow.

In [ ]:
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))


## 1) Responsibilities recap (the "what goes where" rule)

**View (UI)**
- Renders widgets, binds events, displays errors.
- No I/O, no domain logic, no mapping.

**ViewModel (UI state + commands)**
- Holds UI state and handles UI commands.
- Builds *request snapshots* and calls UseCases.
- No network/filesystem I/O, no raw API payloads.

**UseCase**
- Orchestrates workflows and maps errors.
- Depends on Ports, not on concrete adapters.

## 2) Initialization pattern (dependency injection)

A common pattern in this repo is:

1. **Create adapters** (real or mock).
2. **Create use cases** that depend on those ports.
3. **Create a ViewModel** with those use cases.
4. **Create a View** that binds to the ViewModel.

This keeps each layer independent and easy to swap for tests or demos.

In [ ]:
# Step 1: choose adapters (mock for offline training)
from seva.adapters.job_rest_mock import JobRestMock

# Step 2: create use cases
from seva.usecases.build_experiment_plan import BuildExperimentPlan
from seva.usecases.start_experiment_batch import StartExperimentBatch

job_port = JobRestMock()
start_batch_uc = StartExperimentBatch(job_port=job_port)
plan_builder_uc = BuildExperimentPlan()

## 3) A tiny ViewModel example

The ViewModel below does **not** create HTTP payloads and does **not** talk to I/O.
It only:
- Keeps UI state
- Builds snapshot objects
- Calls use cases

In [ ]:
from dataclasses import dataclass
from typing import Tuple

from seva.usecases.build_experiment_plan import (
    ExperimentPlanRequest,
    ModeSnapshot,
    WellSnapshot,
)


@dataclass
class StartRunViewModel:
    build_plan: BuildExperimentPlan
    start_batch: StartExperimentBatch

    experiment_name: str = ""
    wells: Tuple[str, ...] = ()

    def start_clicked(self):
        request = ExperimentPlanRequest(
            experiment_name=self.experiment_name,
            subdir=None,
            client_datetime_override=None,
            wells=self.wells,
            well_snapshots=(
                WellSnapshot(
                    well_id=well_id,
                    modes=(
                        ModeSnapshot(
                            name="CV",
                            params={
                                "cv.start_v": -0.2,
                                "cv.vertex1_v": 0.3,
                                "cv.vertex2_v": -0.3,
                                "cv.final_v": 0.0,
                                "cv.scan_rate_v_s": 0.1,
                                "cv.cycles": 2,
                            },
                        ),
                    ),
                )
                for well_id in self.wells
            ),
        )
        plan = self.build_plan(request)
        return self.start_batch(plan)

## 4) A tiny View example

A View only connects UI events to ViewModel methods. In real code this could be
Tkinter, Qt, or a web UI — the concept stays the same.

In [ ]:
class StartRunView:
    def __init__(self, vm: StartRunViewModel):
        self.vm = vm

    def on_click_start(self):
        # UI event handler → delegate to ViewModel
        result = self.vm.start_clicked()
        print("Run group id:", result.run_group_id)


# Wiring it together
vm = StartRunViewModel(
    build_plan=plan_builder_uc,
    start_batch=start_batch_uc,
)
vm.experiment_name = "Part 2 Demo"
vm.wells = ("A1", "A2")

view = StartRunView(vm)
view.on_click_start()

## 5) How modularity is achieved

Modularity comes from **dependency injection** + **ports**:

- If you swap `JobRestMock` for `JobRestAdapter`, you get a real backend.
- If you swap a UseCase with a fake one, you can test UI behavior without touching the domain.

This keeps UI code *dumb* and stable even when backend APIs change.

In [ ]:
# Swap adapters without changing ViewModel code
from seva.adapters.job_rest import JobRestAdapter
from seva.domain.entities import BoxId

real_job_port = JobRestAdapter(
    base_urls={BoxId("box-1"): "http://localhost:8000"},
    api_keys={BoxId("box-1"): "my-key"},
)

real_start_batch_uc = StartExperimentBatch(job_port=real_job_port)

vm_real = StartRunViewModel(
    build_plan=plan_builder_uc,
    start_batch=real_start_batch_uc,
)

## 6) Exercises

1. Add a new UI field to the ViewModel (e.g., `subdir`) and route it into the request.
2. Replace `JobRestMock` with your own adapter that prints what it would send.
3. Build a fake UseCase that always returns a fixed `run_group_id` and see how the View behaves.